## Deep Convolutional GAN

This exercise is based on the [PyTorch tutorial](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html) on DCGAN.


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## For this exercise it is recommended to use the GPU!

In [0]:

use_cuda = True

if use_cuda and torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

device

device(type='cuda')

### Load the CIFAR10 dataset

Feel free to try other datasets as well, such as the Celeb-A Faces Dataset which is used in the [tutorial](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html) on which this exercise is based.

In [0]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.Scale(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data_cifar', train=True,
                                        download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR10(root='./data_cifar', train=False,
                                       download=True, transform=transform_test)

batch_size = 128

c, w, h = 3, 32, 32

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True)

testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=batch_size,
                                         shuffle=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
0it [00:00, ?it/s]

170500096it [00:09, 18174836.01it/s]                               


Extracting ./data_cifar/cifar-10-python.tar.gz to ./data_cifar
Files already downloaded and verified


### Define the Generator and Discriminator models

In [0]:

n_noise = 100  # Size of the noise vector
n_g_channels = 64
n_d_channels = 64

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(n_noise, n_g_channels * 8, kernel_size=(4, 4), stride=1, padding=0, bias=False),
            nn.BatchNorm2d(n_g_channels * 8),
            nn.ReLU(),
            # state size. (n_g_channels*8) x 4 x 4
            nn.ConvTranspose2d(n_g_channels * 8, n_g_channels * 4, kernel_size=(4, 4), stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_g_channels * 4),
            nn.ReLU(),
            # state size. (n_g_channels*4) x 8 x 8
            nn.ConvTranspose2d(n_g_channels * 4, n_g_channels * 2, kernel_size=(4, 4), stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_g_channels * 2),
            nn.ReLU(),
            # state size. (n_g_channels*2) x 16 x 16
            nn.ConvTranspose2d(n_g_channels * 2,     n_g_channels, kernel_size=(4, 4), stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_g_channels),
            nn.ReLU(),
            # state size. (n_g_channels) x 32 x 32
            nn.ConvTranspose2d(    n_g_channels,      c, kernel_size=(4, 4), stride=2, padding=1, bias=False),
            nn.Tanh()
            # state size. c x 64 x 64
        )

    def forward(self, x):
        return self.model(x)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # input is c x 64 x 64
            nn.Conv2d(c, n_d_channels, kernel_size=(4, 4), stride=2, padding=1, bias=False),
            # nn.LeakyReLU(0.2, inplace=True),
            nn.LeakyReLU(0.2),
            # state size. (n_d_channels) x 32 x 32
            nn.Conv2d(n_d_channels, n_d_channels * 2, kernel_size=(4, 4), stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_d_channels * 2),
            # nn.LeakyReLU(0.2, inplace=True),
            nn.LeakyReLU(0.2),
            # state size. (n_d_channels*2) x 16 x 16
            nn.Conv2d(n_d_channels * 2, n_d_channels * 4, kernel_size=(4, 4), stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_d_channels * 4),
            # nn.LeakyReLU(0.2, inplace=True),
            nn.LeakyReLU(0.2),
            # state size. (n_d_channels*4) x 8 x 8
            nn.Conv2d(n_d_channels * 4, n_d_channels * 8, kernel_size=(4, 4), stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_d_channels * 8),
            # nn.LeakyReLU(0.2, inplace=True),
            nn.LeakyReLU(0.2),
            # state size. (n_d_channels*8) x 4 x 4
            nn.Conv2d(n_d_channels * 8, 1, kernel_size=(4, 4), stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.model(x)
        return output.view(-1, 1)

In [0]:

G = Generator().to(device)
print(G)

D = Discriminator().to(device)
print(D)

Generator(
  (model): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)
Discriminator(
  (model): Sequential(
    (0): 

In [0]:

criterion = nn.BCELoss()

# setup optimizers
D_optimizer = optim.Adam(D.parameters(), lr=0.0001)
G_optimizer = optim.Adam(G.parameters(), lr=0.0001)

### Define a function that displays an image tensor

In [0]:
inverse_transform = transforms.Compose([transforms.Normalize(mean = [ 0., 0., 0. ], std = [ 1/0.229, 1/0.224, 1/0.225 ]), 
                                        transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ], std = [ 1., 1., 1. ]),
                               ])

# Function to show an image tensor
def show(X):
    X = X.cpu()
    X = inverse_transform(X)

    plt.imshow(np.transpose(X.numpy(), (1, 2, 0)))
    plt.show()

### Train the Generator and Discriminator models

In [0]:
start=time.time()

n_show = 16  # Number of generated images that are shown during training

fixed_noise = torch.FloatTensor(batch_size, n_noise, 1, 1).normal_(0, 1).to(device)

for epoch in range(0,200):

  G.train()  # Put the networks in train mode
  D.train()
  for i, (x_batch, y_batch) in enumerate(trainloader):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)  # Move the data to the device that is used

    ############################
    # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
    ###########################
    D_optimizer.zero_grad()

    # Train on real data
    ones_batch = torch.ones(x_batch.size(0), 1).to(device)  # Get a batch of ones indicating these are real images
    output_real = D(x_batch)  # Discriminate the images
    D_real_loss = criterion(output_real, ones_batch)

    # Train on fake data
    noise = torch.normal(mean=0, std=1, size=(batch_size, n_noise, 1, 1)).to(device)  # Create random noise as input for G
    x_batch_fake = G(noise)  # Create fake images using the generator network
    output_fake = D(x_batch_fake.detach())  # Discriminate the images. Detach the tensor for the computation graph so G's parameters won't be optimized here
    zero_batch = torch.zeros(x_batch_fake.size(0), 1).to(device)  # Get a batch of zeros indicating these are fake images
    D_fake_loss = criterion(output_fake, zero_batch)
    
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()

    ############################
    # (2) Update G network: maximize log(D(G(z)))
    ###########################
    G_optimizer.zero_grad()

    ones_batch = torch.ones(x_batch_fake.size(0), 1).to(device)  # Get a batch of ones indicating these are real images
    output_fake = D(x_batch_fake)
    G_loss = criterion(output_fake, ones_batch)
    G_loss.backward()
    G_optimizer.step()

    elapsed = time.time() - start  # Keep track of how much time has elapsed

    if not i % 20:
      print(f'epoch: {epoch}, time: {elapsed:.3f}s, D loss: {D_loss.item():.3f}, G loss: {G_loss.item():.3f}')
    if not i % 100:
      show(torchvision.utils.make_grid(x_batch_fake[:n_show].detach()))





Output hidden; open in https://colab.research.google.com to view.